# EX 5

### EX 1

Using Monte Carlo Methods evaluate the following integral:
$$
\int_{\Omega} \sin \sqrt{\ln (x+y+1)} dx dy
$$
where
$$
\Omega = \left\{ (x,y) : \left( x - \frac{1}{2} \right)^2 + \left( y - \frac{1}{2} \right)^2 \leq \frac{1}{4} \right\}
$$

In [5]:
# we are inside a circle of radius 1/2
# (x,y) are in [-1,0]

to_integrate_tilde <- function(x_tilde,y_tilde) {
val <- sin(sqrt(log(x_tilde/2+y_tilde/2+2)))
return(val)
}

domain_x <- function(u) {
rho <- sqrt(u)
return (rho)
}

domain_y <- function(u) {
theta <- u*2*pi
return (theta)
}


hit_or_miss_1 <- function(Num_points){

M <- 1 #the maximum value of the function
m <- -1 #the minimum value of the function
constant <- (pi*(M-m))/Num_points

x_vals <- domain_x(runif(Num_points))
y_vals <- domain_y(runif(Num_points))
comparison <- runif(Num_points, m, M)

mask <- to_integrate_tilde(x_vals,y_vals)>=0
pos <- sum(ifelse(to_integrate_tilde(x_vals,y_vals)*mask>=comparison*mask, 1, 0))
neg <- sum(ifelse(to_integrate_tilde(x_vals,y_vals)*!mask<=comparison*!mask, 1, 0))
val <- constant*(pos-neg)
return(val)
}

cat("The value of the integral is",hit_or_miss_1(10000))

The value of the integral is -0.2946814

### EX 2

Using Monte Carlo methods, evaluate the volume of the region whose points satisfy the following inequalities:

$$
\begin{cases}
0\leq x \leq 1, \quad 0\leq y \leq 1, \quad 0\leq z \leq 1 \\
x^2+\sin y \leq z\\
x-z+e^y \leq 1
\end{cases}
$$

In [6]:
check <- function(x,y,z) {
val <- ifelse(0<=x & x<=1 & 0<=y & y<=1 & 0<=z & z<=1 & x**2+sin(y)<=z & x-z+exp(y)<=1, 1,0)
return(val)
}

hit_or_miss_2 <- function(Num_points) {
u1 <- runif(Num_points)
u2 <- runif(Num_points)
u3 <- runif(Num_points)
val <- sum(check(u1,u2,u3))/Num_points
return(val)
}

cat("The value of the area is",hit_or_miss_2(10000))

The value of the area is 0.1273

### EX 3

Consider the volume above the cone $z^2=x^2+y^2$ and inside the sphere $x^2+y^2+(z-1)^2=1$. The volume is contained in the box bounded by the inequalities
$$
-1 \leq x \leq 1, \quad -1 \leq y \leq 1, \quad 0\leq z \leq 2
$$

In [7]:
check <- function(x,y,z) {
ifelse( (x**2+y**2+(z-1)**2<=1 & z>=1) | (z**2>=x**2+y**2 & z<1), 1, 0)
}

hit_or_miss_3 <- function(N) {
volume <- 8
xs <- runif(N,-1,1)
ys <- runif(N,-1,1)
zs <- runif(N, 0,2)
res <- sum(check(xs,ys,zs))/N
return(res*volume)
}

cat("The value of the integral is",hit_or_miss_3(10000))

The value of the integral is 3.148

### EX 4

Using the importance sampling method, evaluate the integral
$$
\int_b^\infty x^{\alpha-1} e^{-x} dx
$$
with $\alpha >1$ and $b>0$.

###### (a)
one possibility is to use as sampling function $g(x) = e^{−x}$ in the domain $[b,\infty]$

In [10]:
func <- function(x, a) {
val <- x**(a-1)*exp(-x)
return(val)
}

bad_func <- function(x, b) {
return(exp(-x+b))
}

bad_sampling <- function(b, N) {
val <- b-log(-runif(N)+1)
return(val)
}

with_bad <- function(N, a, b) {
points <- bad_sampling(b, N)
val <- (1/N)*sum(func(points, a=a)/bad_func(points, b))
return(val)
}

integral_with_bad <- function(N,a,b){
ifelse(a<=1 | b<=0,
cat("Error: alpha must be greater than 1 and b must be greater than 0."),
with_bad(N,a,b))
}

alpha <- 1.5
b <- 1
cat("The value of the integral is",integral_with_bad(10000, alpha, b))

The value of the integral is 0.5061826

###### (b)
a more efficient method, especially for large $b$, is to use $g(x) =\lambda e^{−\lambda(x−b)}$

In [11]:
good_func <- function(x, b, l) {
return(l*exp(-l*(x-b)))
}

good_sampling <- function(N,b,l) {
val <- b+(1/l)*log(1/(1-runif(N)))
return(val)
}

with_good <- function(N, a, b, l) {
points <- good_sampling(N, b, l)
val <- (1/N)*sum(func(points, a=a)/good_func(points, b, l))
return(val)
}

#best value
l_star <- function(a,b) {
val <- ((b-a)+sqrt((b-a)**2+4*b))/(2*b)
return(val)
}

integral_with_good <- function(N,a,b, l=l_star(a,b)) {
ifelse(a<=1 | b<=0,
cat("Error: alpha must be greater than 1 and b must be greater than 0."),
with_good(N,a,b,l))
}

alpha <- 1.5
b <- 1
cat("The value of the integral is",integral_with_good(10000, alpha, b))

The value of the integral is 0.5072847

### EX 5

Using the importance sampling method, evaluate the integral
$$
\int_b^\infty x^{\alpha-1} e^{-x} dx
$$
with $\alpha \leq 1$ and $b>0$.
Hint: use the sampling function $g(x) = e^{−(x−b)}$ in the domain $[b,\infty]$

In [12]:
func <- function(x, a) {
val <- x**(a-1)*exp(-x)
return(val)
}

aid_func <- function(x, b) {
return(exp(-x+b))
}

sampling <- function(b, N) {
val <- b-log(-runif(N)+1)
return(val)
}

with_aid <- function(N, a, b) {
points <- sampling(b, N)
val <- (1/N)*sum(func(points, a=a)/aid_func(points, b))
return(val)
}

integral_with_aid <- function(N,a,b){
ifelse(a>1 | b<=0,
cat("Error: alpha must be less or equal than 1 and b must be greater than 0."),
with_aid(N,a,b))
}


alpha <- -1.5
b <- 1
cat("The value of the integral is",integral_with_aid(10000, alpha, b))

The value of the integral is 0.1258823